# **Face Mask Detection 😷**

> 💯MOTTO In this project we are going to make a ```Deep CNN model``` to be able to classify whether or not if a person is wearing a mask or not

In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

### Load & Import helper functions

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

## Loading Data

In [ ]:
BATCH_SIZE=32
IMAGE_SIZE=(224, 224)

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator( 
    rescale=1./255,
    rotation_range=40,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest',
)

train_generator = train_datagen.flow_from_directory('/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Train',
                                                    shuffle=True,
                                                    batch_size = BATCH_SIZE,
                                                    target_size = IMAGE_SIZE,
                                                    class_mode = 'binary',
                                                   )

test_datagen =  tf.keras.preprocessing.image.ImageDataGenerator( 
    rescale=1./255,
)

test_generator = test_datagen.flow_from_directory('/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Test',
                                                    shuffle=False,
                                                    batch_size = BATCH_SIZE,
                                                    target_size = IMAGE_SIZE,
                                                    class_mode = 'binary')

## Visualizing an Image

In [ ]:
class_names = ['mask', 'no mask']
def visualize_images():
    sample_batch = next(test_generator)
    images, labels = sample_batch
    for i in range(5):
        plt.subplot(1, 5, i+1)
        plt.imshow(images[i])
        plt.title(f'{class_names[int(labels[i])]}')
    plt.subplots_adjust(wspace=0.9, hspace=0.9)
        
visualize_images()

## Model Building

* Transfer Learning Techniques

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Dense, Flatten, BatchNormalization, Dropout

from helper_functions import create_tensorboard_callback

In [ ]:
# Creating a Model CheckPoint callback to save the model's weigth only per epoch
def model_checkpoint(checkpoint_file_path):
    callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_file_path,
                                             save_weights_only=True,
                                             save_best_only=True,
                                             save_freq='epoch', # save per epoch
                                             verbose=1)
    return callback

## Inception V3

In [ ]:
from tensorflow.keras.applications import InceptionV3
inceptionv3_convbase = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in inceptionv3_convbase.layers:
    layer.trainable = False # Freezing all the Layers for Feature Extraction

In [ ]:
x = Flatten()(inceptionv3_convbase.output)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)
inceptionv3_model = Model(inceptionv3_convbase.input, outputs)

# Compiling model
inceptionv3_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

# Model Building
history_inceptionv3 = inceptionv3_model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    batch_size=32,
    callbacks=[create_tensorboard_callback(dir_name='Face_Mask_Detection',
                                          experiment_name='InceptionV3')]    
)

### **Inception V3 Model**

In [ ]:
results_inceptionv3_test = inceptionv3_model.evaluate(test_generator)
results_inceptionv3_test

In [ ]:
# Evaluating on Validation Data
val_datagen =  tf.keras.preprocessing.image.ImageDataGenerator( 
    rescale=1./255,
)

val_generator = val_datagen.flow_from_directory('/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Validation',
                                                shuffle=False,
                                                batch_size = BATCH_SIZE,
                                                target_size = IMAGE_SIZE,
                                                class_mode = 'binary')

In [ ]:
results_inceptionv3_val = inceptionv3_model.evaluate(val_generator)
results_inceptionv3_val

In [ ]:
import pandas as pd
pd.DataFrame(history_inceptionv3.history).plot()

🔑 **Note:** We are going to choose InceptionV3 model because its better val_accuracy

## Single Predictions using InceptionV3 model

In [ ]:
def single_predictions(img_path):
    print("Reading image:", img_path)
    encoded_image = tf.io.read_file(img_path)
    image = tf.io.decode_image(encoded_image)

    # Changing the dtype to float32
    image = tf.cast(image, dtype=tf.float32)
    # Normalize the image data to [0, 1]
    image = image / 255.0

    new_height = 224
    new_width = 224
    input_tensor = tf.expand_dims(tf.image.resize(image, [new_height, new_width]), axis=0)

    # Making Predictions
    y_probs = inceptionv3_model.predict(input_tensor)
    y_pred = tf.round(y_probs)
    if y_pred == 0:
        y_probs = 1.0 - y_probs
    plt.imshow(image)  # Convert to NumPy array for displaying
    plt.title(f'{class_names[int(tf.squeeze(y_pred))]} \n Probability : {y_probs}')
    plt.show()

In [ ]:
single_predictions('/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Validation/WithMask/1204.png')

In [ ]:
single_predictions('/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Validation/WithoutMask/1358.png')

In [ ]:
single_predictions('/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Validation/WithMask/Augmented_101_8850689.png')

In [ ]:
single_predictions('/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Validation/WithMask/Augmented_158_8653551.png/')

## Saving InceptionV3 Model

In [ ]:
inceptionv3_model.save('inceptionv3_model.h5')